In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten

%matplotlib inline

Using TensorFlow backend.


In [2]:
data_dir = 'data/miniimagenet/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'train')

In [3]:
IMG_SIZE = 64
IMG_CHANNELS = 3
N_CLASSES = len(os.listdir(train_dir))
N_INNER_CLASSES = 10
INNER_EPOCHS = 1
INNER_BATCH_SIZE = 64
N_INNER_STEPS = 20
INNER_LR = 0.005
OUTER_EPOCHS = 50000
OUTER_LR = 0.01

## Define Image Loading Functions

In [4]:
img_cache = {}

In [5]:
def preprocess_img(img, size=IMG_SIZE):
    return cv2.resize(img, (size, size))

def load_img(class_dir=None, img_name=None, img_path=None):
    if img_path:
        img_name = img_path[img_path.rfind('/')+1:]
    else:
        img_path = os.path.join(class_dir, img_name[:img_name.find('_')], img_name)
    
    # If image is already loaded into cache return it
    if img_name in img_cache:
        return img_cache[img_name]
    else:
        img = cv2.imread(img_path)
        img = preprocess_img(img, size=IMG_SIZE)
        img = img / 255.
        img_cache[img_name] = img
        return img

## Building the DataFrame of Image Names

In [14]:
train_classes = np.asarray(os.listdir(train_dir))

cols = ['class', 'img_name']
df_train = pd.DataFrame(columns=cols)

for cat in train_classes:
    class_list = []
    cat_dir = os.listdir(os.path.join(train_dir, cat))
    for img_name in cat_dir:
        class_list.append([cat, img_name])
    tmp_df = pd.DataFrame(class_list, columns=cols)
    df_train = df_train.append(tmp_df)
    
df_train = df_train.sample(frac=1)
df_train.reset_index(inplace=True)
df_train.drop('index', inplace=True, axis=1)

## Load Validation Set

In [15]:
valid_classes = np.asarray(os.listdir(train_dir))

cols = ['class', 'img_name']
df_valid = pd.DataFrame(columns=cols)

for cat in train_classes:
    class_list = []
    cat_dir = os.listdir(os.path.join(train_dir, cat))
    for img_name in cat_dir:
        class_list.append([cat, img_name])
    tmp_df = pd.DataFrame(class_list, columns=cols)
    df_valid = df_valid.append(tmp_df)
    
df_valid = df_train.sample(frac=1)
df_valid.reset_index(inplace=True)
df_valid.drop('index', inplace=True, axis=1)

In [16]:
df_valid_sample = df_valid.sample(n=10000)

,class,img_name
0,n02108551,n02108551_7511.JPEG
1,n04515003,n04515003_16363.JPEG
2,n03908618,n03908618_226.JPEG
3,n02101006,n02101006_3868.JPEG
4,n04604644,n04604644_2982.JPEG
5,n04612504,n04612504_37418.JPEG
6,n02108915,n02108915_5351.JPEG
7,n03476684,n03476684_15498.JPEG
8,n04509417,n04509417_5180.JPEG
9,n02108089,n02108089_1308.JPEG


## Functions for Generating Tasks and Formatting Data

In [8]:
def sample_task(data, n_classes=N_INNER_CLASSES):
    classes = np.random.choice(train_classes, size=n_classes, replace=False)
    task_indices = data['class'].map(lambda x: x in classes)
    task_data = data[task_indices]
    return task_data

def gen_batches(task_data, batch_size=INNER_BATCH_SIZE, data_dir=train_dir):
    task_data = pd.concat([task_data, pd.get_dummies(task_data['class'])], axis=1)
    while True:
        epoch_data = task_data.sample(frac=1)
        for i in range((len(epoch_data) // batch_size) - 1):
            X_names = epoch_data.iloc[i*batch_size:(i+1)*batch_size]['img_name'].values
            X = np.asarray([load_img(data_dir, img_name) for img_name in X_names])
            y = epoch_data.drop(['class', 'img_name'], axis=1).iloc[i*batch_size:(i+1)*batch_size].values
            
            yield X, y

## Building a Model

In [9]:
model = Sequential()
model.add(InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, IMG_CHANNELS)))
model.add(Conv2D(32, 5, activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(64, 3, activation='relu'))
model.add(Conv2D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(N_INNER_CLASSES, activation='softmax'))

model.compile(optimizer='SGD',
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 43264)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                432650    
Total params: 490,506
Trainable params: 490,506
Non-trainable params: 0
_________________________________________________________________


## Training the Model

In [10]:
for i in range(OUTER_EPOCHS):
    task = sample_task(df_train)
    data = gen_batches(task)
    initial_weights = model.get_weights()
    model.fit_generator(data,
              steps_per_epoch=N_INNER_STEPS,
              epochs=1, 
              verbose=1)
    model.set_weights([iw + OUTER_LR * (cw - iw)
                       for iw, cw in zip(initial_weights, model.get_weights())])

Epoch 1/1
20/20 [==============================] - 12s 613ms/step - loss: 2.3043 - acc: 0.1109
Epoch 1/1
20/20 [==============================] - 16s 789ms/step - loss: 2.2921 - acc: 0.1172
Epoch 1/1
20/20 [==============================] - 11s 554ms/step - loss: 2.3004 - acc: 0.1219
Epoch 1/1
20/20 [==============================] - 10s 505ms/step - loss: 2.2942 - acc: 0.1117
Epoch 1/1
20/20 [==============================] - 12s 585ms/step - loss: 2.2928 - acc: 0.1031
Epoch 1/1
20/20 [==============================] - 11s 527ms/step - loss: 2.2951 - acc: 0.1164
Epoch 1/1
20/20 [==============================] - 10s 522ms/step - loss: 2.3015 - acc: 0.1086
Epoch 1/1
20/20 [==============================] - 11s 546ms/step - loss: 2.2938 - acc: 0.1281
Epoch 1/1
20/20 [==============================] - 8s 391ms/step - loss: 2.2993 - acc: 0.1211
Epoch 1/1
20/20 [==============================] - 8s 397ms/step - loss: 2.2976 - acc: 0.0930
Epoch 1/1
20/20 [==============================] - 1

20/20 [==============================] - 1s 49ms/step - loss: 2.2962 - acc: 0.1172
Epoch 1/1
20/20 [==============================] - 2s 78ms/step - loss: 2.2818 - acc: 0.1484
Epoch 1/1
20/20 [==============================] - 1s 44ms/step - loss: 2.2979 - acc: 0.1078
Epoch 1/1
20/20 [==============================] - 1s 45ms/step - loss: 2.2943 - acc: 0.1164
Epoch 1/1
20/20 [==============================] - 1s 35ms/step - loss: 2.2968 - acc: 0.1086
Epoch 1/1
20/20 [==============================] - 1s 47ms/step - loss: 2.2906 - acc: 0.1477
Epoch 1/1
20/20 [==============================] - 1s 40ms/step - loss: 2.2933 - acc: 0.1305
Epoch 1/1
20/20 [==============================] - 2s 96ms/step - loss: 2.2860 - acc: 0.1117
Epoch 1/1
20/20 [==============================] - 1s 34ms/step - loss: 2.2896 - acc: 0.1492
Epoch 1/1
20/20 [==============================] - 1s 35ms/step - loss: 2.2919 - acc: 0.1414
Epoch 1/1
20/20 [==============================] - 1s 31ms/step - loss: 2.2898 -

20/20 [==============================] - 0s 10ms/step - loss: 2.2959 - acc: 0.1234
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2853 - acc: 0.1523
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2720 - acc: 0.1703
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2715 - acc: 0.1875
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2873 - acc: 0.1461
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2892 - acc: 0.1094
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2793 - acc: 0.1383
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2694 - acc: 0.1836
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2807 - acc: 0.1562
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2876 - acc: 0.1187
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2903 -

20/20 [==============================] - 0s 10ms/step - loss: 2.2771 - acc: 0.1547
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2809 - acc: 0.1500
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2849 - acc: 0.1305
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2763 - acc: 0.1328
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.3004 - acc: 0.1039
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2744 - acc: 0.1398
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2882 - acc: 0.1094
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2763 - acc: 0.1500
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2891 - acc: 0.1242
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2621 - acc: 0.1742
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2827 -

20/20 [==============================] - 0s 10ms/step - loss: 2.2982 - acc: 0.1250
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2817 - acc: 0.1422
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2669 - acc: 0.1711
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2655 - acc: 0.1562
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2952 - acc: 0.1070
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2720 - acc: 0.1734
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2734 - acc: 0.1406
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2793 - acc: 0.1500
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2881 - acc: 0.1187
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2812 - acc: 0.1406
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2731 -

...

20/20 [==============================] - 0s 10ms/step - loss: 2.2693 - acc: 0.1531
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2661 - acc: 0.1500
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2261 - acc: 0.1953
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1139 - acc: 0.2422
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2784 - acc: 0.1453
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2020 - acc: 0.1820
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.3033 - acc: 0.1117
Epoch 1/1
20/20 [==============================] - 0s 12ms/step - loss: 2.2142 - acc: 0.1961
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2672 - acc: 0.1469
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1374 - acc: 0.2391
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2409 -

20/20 [==============================] - 0s 10ms/step - loss: 2.1717 - acc: 0.2352
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2502 - acc: 0.1453
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2432 - acc: 0.1555
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2432 - acc: 0.1695
Epoch 1/1
20/20 [==============================] - 0s 12ms/step - loss: 2.2385 - acc: 0.1641
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2143 - acc: 0.1625
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2326 - acc: 0.1797
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1611 - acc: 0.2023
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2292 - acc: 0.1898
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2273 - acc: 0.1898
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2416 -

20/20 [==============================] - 0s 10ms/step - loss: 2.2526 - acc: 0.1609
Epoch 1/1
20/20 [==============================] - 0s 12ms/step - loss: 2.2498 - acc: 0.1609
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2763 - acc: 0.1375
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2385 - acc: 0.1633
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.1844 - acc: 0.2086
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.1790 - acc: 0.1695
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.0299 - acc: 0.2641
Epoch 1/1
20/20 [==============================] - 0s 12ms/step - loss: 2.2111 - acc: 0.1758
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1539 - acc: 0.1977
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2431 - acc: 0.1391
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2235 -

20/20 [==============================] - 0s 10ms/step - loss: 2.2493 - acc: 0.1477
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1265 - acc: 0.2148
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2426 - acc: 0.1820
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1860 - acc: 0.2023
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2672 - acc: 0.1523
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1004 - acc: 0.2156
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1341 - acc: 0.2258
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2745 - acc: 0.1359
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2855 - acc: 0.1242
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2153 - acc: 0.1719
Epoch 1/1
20/20 [==============================] - 0s 11ms/step - loss: 2.2057 -

20/20 [==============================] - 0s 10ms/step - loss: 2.0789 - acc: 0.2391
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2127 - acc: 0.1906
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1946 - acc: 0.1891
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1330 - acc: 0.2258
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1612 - acc: 0.2086
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.2525 - acc: 0.1680
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1860 - acc: 0.1828
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1394 - acc: 0.2125
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.0545 - acc: 0.2398
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1726 - acc: 0.2109
Epoch 1/1
20/20 [==============================] - 0s 10ms/step - loss: 2.1045 -